In [75]:
import pandas as pd
import numpy as np
import geopandas as gpd
import leafmap.plotlymap as leafmap
import dash_leaflet as dl #from tutorial, replace with leafmap later
#dcc = dash_core_components
#hmtl = dash_html_components
from dash import Dash, html, dcc, callback, Output, Input, dash_table
import plotly.express as px


In [76]:
geom1_gdf = gpd.read_file('data/terrawise/Stratification/1/cea part/cea_1.shp')
geom2_gdf = gpd.read_file('data/terrawise/Stratification/2/cea part/cea_2.shp')
geom3_gdf = gpd.read_file('data/terrawise/Stratification/3/cea part/cea_3.shp')


In [77]:
"""
label - assume 123 correspond to cea 1,2,3
year - will need to be able to filter by year
For a given year, map should show areas where a practice was/ was not implemented.
"""
practice_csv = pd.read_csv('data/terrawise/terrawise-practice-info.csv')



In [87]:

app = Dash(__name__)

app.layout = html.Div([
    html.Div(children='Practice management data'),
    html.Hr(),
    dcc.RadioItems(options=['Urea', 'Synthetic_fertiliser'], value='Urea', id='controls-and-radio-item'),
    dash_table.DataTable(data=practice_csv.to_dict('records'), page_size=5),
    dcc.Graph(figure={}, id='controls-and-graph'),
    html.Div([
        dl.Map(children=[
            dl.TileLayer(),
            dl.GeoJSON(id='shapes')],
        id='map',
        center=[-31,122], #replace with the centroid of the imported geometry
        zoom=4, #bounding box of geometry rather than coded zoom
        style={'height': '50vh'})
        ])
])

@callback(
    Output(component_id='controls-and-graph', component_property='figure'),
    Input(component_id='controls-and-radio-item', component_property='value')
)

def update_graph(col_chosen):
    fig = px.histogram(practice_csv, x=col_chosen, y='label', histfunc='count')
    return fig

if __name__ == '__main__':
    app.run(debug=True, port=8051)